# Benchmark classical approaches

In [1]:
import os
import sys
import nltk
from tqdm import trange
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
from src.preprocessing.corpus_preprocessor import CorpusPreprocessor
from src.preprocessing.functions import *
from src.preprocessing.consts import EMAIL_REGEX
from src.ds_loaders.xsum import XSumLoader
from src.ds_loaders.cnn_daily import CNNDailyLoader
from src.metrics import BLEU, ROUGE
from src.metrics.GloVeCosineSimilarity import GloveCosineSimilarity
from src.model import *
import math

[nltk_data] Downloading package punkt to /home/pasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pasha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pasha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/pasha/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2022-05-31 08:45:48.571474: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt


## Load dataset

In [2]:
# loader = XSumLoader()
loader = CNNDailyLoader()
loader.load()

Reusing dataset cnn_dailymail (/home/pasha/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# orig_documents, orig_summaries = loader.X_val, loader.y_val
CLIP = 500
orig_documents, orig_summaries = loader.X_test[-CLIP:], loader.y_test[-CLIP:]
print(len(orig_summaries))

500


In [6]:
n_sentences = 0
for y in trange(len(orig_summaries)):
    n_sentences += len(nltk.sent_tokenize(orig_summaries[y]))
n_sentences /= len(orig_summaries)
print(f"Average number of sentences: {n_sentences}")
n_sen = math.ceil(n_sentences)
print(f"Choosing number of sentences: {n_sen}")

100%|██████████| 500/500 [00:00<00:00, 8284.72it/s]

Average number of sentences: 3.94
Choosing number of sentences: 4


## Preprocess dataset

In [7]:
# preprocessor = CorpusPreprocessor(verbose=False).\
#     add(to_lower()).\
#     add(expand_contractions()).\
#     add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
#     add(remove_by_regex([r"\d"])).\
#     add(remove_symbols(string.punctuation.replace('.', ''))).\
#     add(lemmatize())
#     # add(remove_words(set(stopwords.words('english'))))


In [8]:
# documents = []
# summaries = []
# for idx in trange(len(orig_documents)):
#     doc = orig_documents[idx]
#     summ = orig_summaries[idx]
#     if len(doc) == 0 or len(summ) == 0:
#         continue
#     documents.append(preprocessor.transform(doc))
#     summaries.append(preprocessor.transform(summ))

## Create models and metrics and preprocessors

In [9]:
# 1. models
summary_size = n_sen
TextRank = TextRankModel(summary_size)
LexRank = LexRankModel(summary_size)
Lsa = LsaModel(summary_size)
Luhn = LuhnModel(summary_size)
KL = KLModel(summary_size)
MyTextRank = MyTextRankModel(summary_size)

models = [ TextRank, LexRank, Lsa, Luhn, KL, MyTextRank ]

DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt


In [10]:
CLIP = 100
orig_documents, orig_summaries = loader.X_test[-CLIP:], loader.y_test[-CLIP:]
print(len(orig_summaries))

100


In [26]:
for i in range(len(orig_summaries)):
    if "nose" in orig_summaries[i] and "bloodied" in orig_summaries[i]:
        print(i)

print(orig_summaries[92])
i = 92
doc = preprocessor_no.transform(orig_documents)
LexRank.fit(doc)
print(LexRank.predict(doc)[92])


92
Chelsea beat Stoke 2-1 in Premier League at Stamford Bridge on Saturday .
Cesc Fabregas' nose was left bloodied after tussle with Charlie Adam .
Midfielder showed off injury as he posted picture to Instagram after game .
Fabregas joked that he might finally be able to get his nose fixed .


100%|██████████| 100/100 [00:02<00:00, 39.50it/s]

Cesc Fabregas showed off the result of a nasty-looking clash with Charlie Adam's arm during Chelsea's victory against Stoke City at Stamford Bridge on Saturday. The former Barcelona midfielder was left bloodied after being caught by a flailing arm following a tussle with Adam, the scorer of a 66-yard wonder goal. Spain international Fabregas also took a whack on his leg during the clash on Saturday . He avoided having an operation on the injury then, but it seems that Fabregas might be happy to have corrective surgery this time.


In [103]:
# 2. metrics
bleu1 = BLEU(1)
bleu2 = BLEU(2)
bleu3 = BLEU(3)
bleu4 = BLEU(4)

rouge1_f1 = ROUGE("rouge1", "fmeasure")
rouge2_f1 = ROUGE("rouge2", "fmeasure")
rougeL_f1 = ROUGE("rougeL", "fmeasure")
rouge1_r = ROUGE("rouge1", "recall")
rouge2_r = ROUGE("rouge2", "recall")
rougeL_r = ROUGE("rougeL", "recall")
glove_cosine = GloveCosineSimilarity()

metrics = [ bleu1, bleu2, bleu3, bleu4,
            rouge1_f1, rouge2_f1, rougeL_f1, rouge1_r, rouge2_r, rougeL_r ]

DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt
DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt


In [18]:
# 3. preprocessors
preprocessor_strong = CorpusPreprocessor(name="strong").\
    add(to_lower()).\
    add(expand_contractions()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_by_regex([r"\d"])).\
    add(remove_symbols(string.punctuation.replace('.', ''))).\
    add(lemmatize()).\
    add(remove_words(set(stopwords.words('english'))))

preprocessor_mid = CorpusPreprocessor(name="mid").\
    add(to_lower()).\
    add(expand_contractions()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_by_regex([r"\d"])).\
    add(remove_symbols(string.punctuation.replace('.', '')))

preprocessor_simple = CorpusPreprocessor(name="simple").\
    add(to_lower()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_symbols(string.punctuation.replace('.', '')))

preprocessor_no = CorpusPreprocessor(name="nothing")
preprocessors = [preprocessor_strong, preprocessor_mid, preprocessor_simple, preprocessor_no]

In [105]:
result = {}
def evaluate_models(preprocessors,
                    models,
                    metrics,
                    documents,
                    summaries):
    n_jobs = len(models) * len(preprocessors)
    n_done = 0
    for prep in preprocessors:
        result[str(prep)] = {}
        print(f"Starting with preprocessor {prep}")
        prep_docs = prep.transform(documents)
        prep_sum = prep.transform(summaries)
        for model in models:
            result[str(prep)][str(model)] = {}
            print(f"    Model: {model}")
            model.fit(prep_docs)
            pred = model.predict(prep_docs)
            # print(f"        ", end="")
            for m_idx, metric in enumerate(metrics):
                print(f"      {metric}... ", end="")
                metric.reset_state()
                metric.update_state(pred, prep_sum)
                value = metric.result()
                result[str(prep)][str(model)][str(metric)] = value
                print(f"{value} - {m_idx+1} / {len(metrics)}")
            print()
            n_done += 1
            print(f"Done: {n_done}/{n_jobs}")
    return result

In [106]:
models = [ TextRank, LexRank, Lsa, Luhn, KL]
# metrics = [glove_cosine]
# models = [ MyTextRank ]
evaluate_models(
    preprocessors,
    models,
    metrics,
    orig_documents,
    orig_summaries
)

Starting with preprocessor CourpusPreprocessor(strong)
    Model: TextRank


100%|██████████| 500/500 [00:07<00:00, 69.56it/s]


      BLEU-1... 0.20843275625729807 - 1 / 10
      BLEU-2... 0.13078957308579928 - 2 / 10
      BLEU-3... 0.08758466538868595 - 3 / 10
      BLEU-4... 0.06302689979364826 - 4 / 10
      ROUGE-1-fmeasure... 0.2802082008853045 - 5 / 10
      ROUGE-2-fmeasure... 0.11552971434225888 - 6 / 10
      ROUGE-L-fmeasure... 0.20241138735028996 - 7 / 10
      ROUGE-1-recall... 0.4753477954330174 - 8 / 10
      ROUGE-2-recall... 0.1986445181977855 - 9 / 10
      ROUGE-L-recall... 0.3441145438091803 - 10 / 10

Done: 1/20
    Model: LexRank


100%|██████████| 500/500 [00:07<00:00, 65.67it/s]


      BLEU-1... 0.2591295477728664 - 1 / 10
      BLEU-2... 0.15825142289863528 - 2 / 10
      BLEU-3... 0.10309360206906006 - 3 / 10
      BLEU-4... 0.07188049423948577 - 4 / 10
      ROUGE-1-fmeasure... 0.3103108999394126 - 5 / 10
      ROUGE-2-fmeasure... 0.12033220121757199 - 6 / 10
      ROUGE-L-fmeasure... 0.214927126176479 - 7 / 10
      ROUGE-1-recall... 0.439371647786182 - 8 / 10
      ROUGE-2-recall... 0.17480503305651524 - 9 / 10
      ROUGE-L-recall... 0.30650915367840303 - 10 / 10

Done: 2/20
    Model: Lsa


...

100%|██████████| 500/500 [00:03<00:00, 139.79it/s]


      BLEU-1... 0.19660066847524693 - 1 / 10
      BLEU-2... 0.11956369604291492 - 2 / 10
      BLEU-3... 0.0833812324641324 - 3 / 10
      BLEU-4... 0.06305080530831957 - 4 / 10
      ROUGE-1-fmeasure... 0.30311503711518895 - 5 / 10
      ROUGE-2-fmeasure... 0.12078476411561606 - 6 / 10
      ROUGE-L-fmeasure... 0.19528758468288718 - 7 / 10
      ROUGE-1-recall... 0.533303267067496 - 8 / 10
      ROUGE-2-recall... 0.21007000103898016 - 9 / 10
      ROUGE-L-recall... 0.3448695703836534 - 10 / 10

Done: 19/20
    Model: KL


100%|██████████| 500/500 [00:38<00:00, 12.82it/s]


      BLEU-1... 0.21271945122660157 - 1 / 10
      BLEU-2... 0.12129901016587288 - 2 / 10
      BLEU-3... 0.0807286740122743 - 3 / 10
      BLEU-4... 0.05930730836589287 - 4 / 10
      ROUGE-1-fmeasure... 0.2849468612411222 - 5 / 10
      ROUGE-2-fmeasure... 0.1014592234994447 - 6 / 10
      ROUGE-L-fmeasure... 0.18605288087151964 - 7 / 10
      ROUGE-1-recall... 0.38293574650558915 - 8 / 10
      ROUGE-2-recall... 0.13599624959820433 - 9 / 10
      ROUGE-L-recall... 0.2516078710291307 - 10 / 10

Done: 20/20


{'CourpusPreprocessor(strong)': {'TextRank': {'BLEU-1': 0.20843275625729807,
   'BLEU-2': 0.13078957308579928,
   'BLEU-3': 0.08758466538868595,
   'BLEU-4': 0.06302689979364826,
   'ROUGE-1-fmeasure': 0.2802082008853045,
   'ROUGE-2-fmeasure': 0.11552971434225888,
   'ROUGE-L-fmeasure': 0.20241138735028996,
   'ROUGE-1-recall': 0.4753477954330174,
   'ROUGE-2-recall': 0.1986445181977855,
   'ROUGE-L-recall': 0.3441145438091803},
  'LexRank': {'BLEU-1': 0.2591295477728664,
   'BLEU-2': 0.15825142289863528,
   'BLEU-3': 0.10309360206906006,
   'BLEU-4': 0.07188049423948577,
   'ROUGE-1-fmeasure': 0.3103108999394126,
   'ROUGE-2-fmeasure': 0.12033220121757199,
   'ROUGE-L-fmeasure': 0.214927126176479,
   'ROUGE-1-recall': 0.439371647786182,
   'ROUGE-2-recall': 0.17480503305651524,
   'ROUGE-L-recall': 0.30650915367840303},
  'Lsa': {'BLEU-1': 0.191573283180524,
   'BLEU-2': 0.11477275996718318,
   'BLEU-3': 0.07438709094008615,
   'BLEU-4': 0.05222961275109926,
   'ROUGE-1-fmeasure': 0.

In [107]:
from pprint import pprint
import json
pprint(result)
with open("bench_results_cnn_daily_last.json", 'w') as f:
    json.dump(result, f, indent=4)

{'CourpusPreprocessor(mid)': {'KL': {'BLEU-1': 0.2547491475888943,
                                     'BLEU-2': 0.1485146598939891,
                                     'BLEU-3': 0.09903989492993166,
                                     'BLEU-4': 0.07257330707603102,
                                     'ROUGE-1-fmeasure': 0.2909479784435467,
                                     'ROUGE-1-recall': 0.3736624390956721,
                                     'ROUGE-2-fmeasure': 0.10294361262245369,
                                     'ROUGE-2-recall': 0.13233878477240044,
                                     'ROUGE-L-fmeasure': 0.19411632643997556,
                                     'ROUGE-L-recall': 0.25111516891854235},
                              'LexRank': {'BLEU-1': 0.24234428289355592,
                                          'BLEU-2': 0.1454105238410503,
                                          'BLEU-3': 0.09978739377071438,
                                          'BLEU-4':

In [13]:
# small_ds = [orig_documents[0], orig_documents[1]]
# small_ds_prep = preprocessor_simple.transform(small_ds)
# # print(small_ds_prep)
#
# # bads = string.punctuation.replace('.', '')
# # print(bads)
# #
# # tmp = remove_symbols(bads)(orig_documents[1])
# MyTextRank.fit(small_ds_prep)
# res = MyTextRank.predict(small_ds_prep)
# # print(tmp)
# # print("=" * 30)
# # print(small_ds_prep[1])
# # print("=" * 30)
# print(len(orig_documents[1].split(".")), len(nltk.sent_tokenize(orig_documents[1])))
# print("=" * 30)
# print(res[1])

100%|██████████| 2/2 [00:00<00:00, 103.39it/s]

12 8
the 37yearold has scored 230 runs in four firstclass games this season at an average of 57.50.
losing adam is naturally a blow as he contributes significantly to everything we do director of cricket angus fraser said.
